In [ ]:
from DBSCANSupport import DBSCANSupport
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
from SeamountHelp import *

In [ ]:
grav_dat = Path('data') / 'test_curv_32.highpass.csv'
grav = pd.read_csv(grav_dat, names=['Longitude', 'Latitude', 'Intensity'], header=0)
grav = grav[['Longitude', 'Latitude', 'Intensity']]
old_train_bounds = (-4.525, -2.38, -95.85833, -93.00833)

In [ ]:
grav["Latitude"].describe()

In [ ]:
grav["Longitude"].describe()

In [ ]:
new_bounds = (-3, 0, -99, -96)
data = DBSCANSupport.formatData(grav, "Intensity")
best_eps = (0.32052631578947366,)
best_samp = (13,)
tester = DBSCANSupport(Path('data') / "sample_mask.txt.xlsx", train_zone=new_bounds)

In [ ]:
tester.addTrainingData(data)
_, _, data_out, out = tester.gridSearch(best_eps, best_samp, verbose=True)

In [ ]:
dfout = pd.DataFrame(data_out, columns=["Lat", "Lon", "Label", "Intensity", "TrueLabel"])
seamount_points = tester.getSeamountPoints()
df_labeled = dfout[(dfout["Label"] != dfout["Label"].mode().iloc[0])]

In [ ]:
test_zone=new_bounds
seamounts = pd.read_excel(Path('data') / "sample_mask.txt.xlsx", \
                              sheet_name="new mask")
seamounts = seamounts.drop(columns=["VGG Height", "Radius", "base_depth", "-",
                                        "Name", "Charted", "surface_depth"])
seamounts = seamounts[(seamounts["Latitude"] >= test_zone[0]) & (seamounts["Latitude"] <= test_zone[1]) &
                          (seamounts["Longitude"] >= test_zone[2]) & (seamounts["Longitude"] <= test_zone[3])]
seamounts = seamounts.to_numpy()

In [ ]:
fig = px.scatter(x=seamounts[:, 0], y=seamounts[:, 1], width=1000, height=500)
fig.add_trace(px.scatter(df_labeled, x="Lon", y="Lat", color="TrueLabel",).data[0])
fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )
fig.update_layout(title="Model labels")
# fig.show()

In [ ]:
raw_test_zone = grav[
    (
        grav["Latitude"] <= new_bounds[1]
    ) & (
        grav["Latitude"] >= new_bounds[0]
    ) & (
        grav["Longitude"] <= new_bounds[3]
    ) & (
        grav["Longitude"] >= new_bounds[2]
    )]

In [ ]:
fig2 = px.scatter(raw_test_zone, x='Longitude', y='Latitude', color='Intensity')
fig2.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )
# fig2.show()

In [ ]:
larger_bounds = (3.574, 8.654, -104.077, -93.371)
with open(Path('data') / 'curv_32.highpass.xyz', 'r') as fin:
    newbound_data = readCroppedxyz(fin, larger_bounds)

In [ ]:
newbound_data

In [ ]:
newbound_data = pd.DataFrame(newbound_data, columns=["Longitude", "Latitude", "Intensity"])
newbound_data = newbound_data[["Latitude", "Longitude", "Intensity"]]

In [ ]:
raw_newbound = px.scatter(newbound_data, x="Longitude", y="Latitude", color="Intensity")
raw_newbound.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )

In [ ]:
new_labels = tester.testData(newbound_data.to_numpy())

In [ ]:
dfout = pd.DataFrame(new_labels, columns=["Latitude", "Longitude", "Label", "Intensity"])
df_labeled = dfout[(dfout["Label"] != dfout["Label"].mode().iloc[0])]
dfout = dfout.to_numpy()

In [ ]:
plotData(df_labeled)

In [ ]:
newtest = DBSCANSupport(Path('data') / "sample_mask.txt.xlsx", train_zone=larger_bounds)
test_trainer = newbound_data[["Latitude", "Longitude", "Intensity"]]
test_trainer = newtest.formatData(test_trainer, "Intensity")
newtest.addTrainingData(test_trainer)
true_lables = newtest.training_data[:, 3]

In [ ]:
lableled_out = np.insert(new_labels, 4, true_lables, axis=1)
lableled_out = pd.DataFrame(lableled_out, columns=["Latitude", "Longitude", "Lables", "Intensity", "TrueLabel"])

In [ ]:
%load_ext autoreload
%autoreload 2
#plotData(lableled_out, "TrueLabel")